In [2]:
%matplotlib inline

### Neural Networks

用torch.nn包构建神经网络。``autograd``，``nn``包依赖``autograd``包定义模型并求导。

一个``nn.Module``包含各个层和``forward(input)``方法，返回``output``。

<img src=https://pytorch.org/tutorials/_images/mnist.png width=800>

一个简单的前馈神经网络，接受一个输入，一层接着一层传递，最后输出计算结果。

神经网络的典型训练过程如下：

1. 定义包含一些可学习参数(权重)的神经网络模型； 
2. 在数据集上迭代； 
3. 通过神经网络处理输入； 
4. 计算损失(输出结果和正确值差值)；
5. 梯度反向传播回网络的参数； 
6. 更新网络参数，使用如下简单的更新原则： 
``weight = weight - learning_rate * gradient``

### 定义网络

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        # If size is a square, can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

    def num_flat_features(self, x):
        # all dimensions except batch dimension
        size = x.size()[1:]  
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)
# print(net.num_flat_features)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


模型必须定义``forward`` 函数，``backward``函数（用来计算梯度）被``autograd``自动创建。

可以在 ``forward`` 函数中使用任何针对 Tensor的操作。``net.parameters()``返回可被学习的参数（权重）列表和值

In [2]:
params = list(net.parameters())
# print(params)
print(len(params))
# conv1's .weight
print(params[0].size())  

10
torch.Size([6, 1, 5, 5])


测试随机输入32×32。
网络（LeNet）期望的输入大小32×32，如果用MNIST数据集训练这个网络，把图片大小重新调整到32×32。

In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0163, -0.1193, -0.1182, -0.1382,  0.0039,  0.0278,  0.0366,  0.0915,
          0.0009, -0.1215]], grad_fn=<AddmmBackward>)


所有参数的梯度缓存清零，进行随机梯度的的反向传播：

In [4]:
net.zero_grad()
out.backward(torch.randn(1, 10))

``torch.nn``只支持小批量输入。整个 ``torch.nn``包都只支持小批量样本，不支持单个样本。

例如``nn.Conv2d`` 接受一个4维张量，每一维分别是``sSamples * nChannels * Height * Width（样本数*通道数*高*宽）``

如果单个样本，使用 ``input.unsqueeze(0)`` 添加其它的维数


**回顾:**
  -  ``torch.Tensor``：一个自动调用``backward()``实现支持自动梯度计算的*多维数组* ，保存关于这个向量的*梯度* 
  -  ``nn.Module``：神经网络模块。封装参数、移动到GPU运行、导出、加载等。
  -  ``nn.Parameter``：当赋值给一个``Module``时，*自动*注册为一个参数。
  -  ``autograd.Function``：实现自动求导操作的前向和反向定义，每个变量操作至少创建一个函数节点。每一个``Tensor``操作都创建一个接到创建``Tensor``和 *编码其历史* 函数的``Function``节点。

**重点如下：**
  -  定义一个网络
  -  处理输入，调用backword

**还剩：**
  -  计算损失
  -  更新网络权重

### 损失函数
一个损失函数接受一对 (output, target) 为输入，估计网络的输出和目标值相差多少。output为网络输出，target为实际值。

nn包中有很多不同的[损失函数](https://pytorch.org/docs/nn.html#loss-functions)。
``nn.MSELoss``是一个比较简单的损失函数，计算输出和目标的**均方误差**。

In [8]:
output = net(input)

# 随机值作为样例
target = torch.randn(10)

# target和output的shape相同
target = target.view(1, -1)
print(target.size())
print(output.size())

criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

torch.Size([1, 10])
torch.Size([1, 10])
tensor(0.4730, grad_fn=<MseLossBackward>)


如果反向过程中跟随``loss``，用``.grad_fn`` 属性，将看到如下所示的计算图。

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

调用 ``loss.backward()``,整张计算图根据loss微分，

图中所有设置为``requires_grad=True``的张量会拥有一个随梯度累积的``.grad`` 张量。

向后退几步:

In [13]:
# MSELoss
print(loss.grad_fn)

# Linear
print(loss.grad_fn.next_functions[0][0])
# ReLU
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

### 反向传播
调用loss.backward()获得反向传播误差。在调用前要清除已存在的梯度，否则梯度被累加到已存在的梯度。

调用loss.backward()，查看conv1层的偏差（bias）项在反向传播前后的梯度。

In [14]:
# 清除梯度
net.zero_grad()  

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0031, -0.0051, -0.0069,  0.0038, -0.0028, -0.0052])


如何使用损失函数: `nn`包，包含各种构成深度神经网络构建模块和损失函数，完整文档查看[here](https://pytorch.org/docs/nn)。

**剩下的最后一件事:**
  - 新网络的权重

### 更新权重

实践中最简单的权重更新规则是随机梯度下降（SGD）：``weight = weight - learning_rate * gradient``

可以使用简单的Python代码实现：
```python
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
```
神经网络想要用各种不同更新规则时，比如SGD、Nesterov-SGD、Adam、RMSPROP等，

PyTorch包``torch.optim``实现所有规则。使用它们非常简单：

In [15]:
import torch.optim as optim

# create optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in training loop:
# zero the gradient buffers
optimizer.zero_grad()
output = net(input)

loss = criterion(output, target)
loss.backward()

# Does the update
optimizer.step()

如何使用``optimizer.zero_grad()``手动将梯度缓冲区设置为零。梯度按Backprop部分的说明累积。